# Solving the Time-Independent Schrödinger Equation Using Finite Differences

## Introduction

The Schrödinger equation is the fundamental equation of quantum mechanics, describing how the quantum state of a physical system evolves. In this notebook, we solve the **time-independent Schrödinger equation** (TISE) for a particle in a one-dimensional potential using the **finite difference method**.

## Theoretical Background

### The Time-Independent Schrödinger Equation

The one-dimensional TISE is given by:

$$\hat{H}\psi(x) = E\psi(x)$$

where the Hamiltonian operator $\hat{H}$ is:

$$\hat{H} = -\frac{\hbar^2}{2m}\frac{d^2}{dx^2} + V(x)$$

This gives us the eigenvalue equation:

$$-\frac{\hbar^2}{2m}\frac{d^2\psi(x)}{dx^2} + V(x)\psi(x) = E\psi(x)$$

### Finite Difference Approximation

We discretize space into $N$ points with spacing $\Delta x$. The second derivative is approximated using the central difference formula:

$$\frac{d^2\psi}{dx^2}\bigg|_{x_i} \approx \frac{\psi_{i+1} - 2\psi_i + \psi_{i-1}}{(\Delta x)^2}$$

### Matrix Formulation

Substituting into the TISE and rearranging, we obtain a matrix eigenvalue problem:

$$\mathbf{H}\boldsymbol{\psi} = E\boldsymbol{\psi}$$

The Hamiltonian matrix has elements:

$$H_{ii} = \frac{\hbar^2}{m(\Delta x)^2} + V(x_i)$$

$$H_{i,i\pm 1} = -\frac{\hbar^2}{2m(\Delta x)^2}$$

This is a tridiagonal matrix that can be efficiently diagonalized to find the energy eigenvalues $E_n$ and eigenfunctions $\psi_n(x)$.

### Natural Units

For computational convenience, we work in **atomic units** where $\hbar = m = 1$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh_tridiagonal

# Set up plotting style
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 12

## Numerical Implementation

### Define the Spatial Grid and Potential

We consider a **quantum harmonic oscillator** potential:

$$V(x) = \frac{1}{2}m\omega^2 x^2$$

In atomic units with $\omega = 1$:

$$V(x) = \frac{1}{2}x^2$$

The analytical energy eigenvalues are:

$$E_n = \hbar\omega\left(n + \frac{1}{2}\right) = n + \frac{1}{2}$$

In [ ]:
# Physical parameters (atomic units: hbar = m = 1)
hbar = 1.0
m = 1.0
omega = 1.0  # Angular frequency for harmonic oscillator

# Spatial grid parameters
x_min = -10.0  # Left boundary
x_max = 10.0   # Right boundary
N = 500        # Number of grid points

# Create spatial grid
x = np.linspace(x_min, x_max, N)
dx = x[1] - x[0]  # Grid spacing

print(f"Grid spacing: dx = {dx:.6f}")
print(f"Number of grid points: N = {N}")

In [ ]:
# Define the potential function
def harmonic_potential(x, omega=1.0):
    """Quantum harmonic oscillator potential: V(x) = 0.5 * m * omega^2 * x^2"""
    return 0.5 * m * omega**2 * x**2

# Calculate potential at each grid point
V = harmonic_potential(x, omega)

# Visualize the potential
plt.figure(figsize=(10, 4))
plt.plot(x, V, 'b-', linewidth=2)
plt.xlabel('Position x (a.u.)')
plt.ylabel('V(x) (a.u.)')
plt.title('Harmonic Oscillator Potential')
plt.grid(True, alpha=0.3)
plt.xlim(x_min, x_max)
plt.ylim(0, 20)
plt.show()

## Constructing the Hamiltonian Matrix

The finite difference Hamiltonian is a tridiagonal matrix. We use `scipy.linalg.eigh_tridiagonal` for efficient diagonalization.

In [ ]:
# Kinetic energy prefactor
kinetic_prefactor = hbar**2 / (2 * m * dx**2)

# Main diagonal: kinetic + potential
main_diag = 2 * kinetic_prefactor + V

# Off-diagonal elements (constant)
off_diag = -kinetic_prefactor * np.ones(N - 1)

print(f"Kinetic prefactor: {kinetic_prefactor:.6f}")
print(f"Main diagonal range: [{main_diag.min():.3f}, {main_diag.max():.3f}]")

## Solving the Eigenvalue Problem

We diagonalize the Hamiltonian matrix to obtain energy eigenvalues and wavefunctions.

In [ ]:
# Solve the tridiagonal eigenvalue problem
energies, wavefunctions = eigh_tridiagonal(main_diag, off_diag)

# Normalize wavefunctions (they should be orthonormal, but we ensure proper normalization)
for i in range(wavefunctions.shape[1]):
    norm = np.sqrt(np.trapz(wavefunctions[:, i]**2, x))
    wavefunctions[:, i] /= norm

# Display first few energy levels
n_states = 6
print("\nEnergy Eigenvalues (first {} states):".format(n_states))
print("="*50)
print(f"{'n':<5} {'E_numerical':<15} {'E_analytical':<15} {'Error':<15}")
print("-"*50)

for n in range(n_states):
    E_analytical = omega * (n + 0.5)  # Exact result for harmonic oscillator
    E_numerical = energies[n]
    error = abs(E_numerical - E_analytical)
    print(f"{n:<5} {E_numerical:<15.6f} {E_analytical:<15.6f} {error:<15.2e}")

## Visualization of Results

We plot the potential, energy levels, and probability densities $|\psi_n(x)|^2$ for the first several eigenstates.

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Color scheme for different states
colors = plt.cm.viridis(np.linspace(0, 0.9, n_states))

# Plot 1: Energy levels in potential well
ax1 = axes[0, 0]
ax1.plot(x, V, 'k-', linewidth=2, label='V(x)')

for n in range(n_states):
    E = energies[n]
    # Draw energy level line
    # Find classical turning points
    mask = V <= E
    if np.any(mask):
        x_left = x[mask][0]
        x_right = x[mask][-1]
        ax1.hlines(E, x_left, x_right, colors=colors[n], linewidth=2)
        ax1.text(x_right + 0.3, E, f'n={n}', fontsize=10, va='center')

ax1.set_xlabel('Position x (a.u.)')
ax1.set_ylabel('Energy (a.u.)')
ax1.set_title('Energy Levels in Harmonic Potential')
ax1.set_xlim(-8, 10)
ax1.set_ylim(0, 8)
ax1.grid(True, alpha=0.3)

# Plot 2: Wavefunctions
ax2 = axes[0, 1]
for n in range(n_states):
    psi = wavefunctions[:, n]
    # Scale and offset for visibility
    scale = 1.5
    ax2.plot(x, scale * psi + energies[n], color=colors[n], linewidth=1.5, label=f'n={n}')
    ax2.axhline(y=energies[n], color=colors[n], linestyle='--', alpha=0.3)

ax2.plot(x, V, 'k-', linewidth=2, alpha=0.5)
ax2.set_xlabel('Position x (a.u.)')
ax2.set_ylabel('ψ(x) + E_n (a.u.)')
ax2.set_title('Wavefunctions (offset by energy)')
ax2.set_xlim(-8, 8)
ax2.set_ylim(-0.5, 8)
ax2.legend(loc='upper right', fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Probability densities
ax3 = axes[1, 0]
for n in range(n_states):
    prob_density = wavefunctions[:, n]**2
    ax3.plot(x, prob_density, color=colors[n], linewidth=1.5, label=f'n={n}')

ax3.set_xlabel('Position x (a.u.)')
ax3.set_ylabel('|ψ(x)|² (a.u.)')
ax3.set_title('Probability Densities')
ax3.set_xlim(-6, 6)
ax3.legend(loc='upper right', fontsize=9)
ax3.grid(True, alpha=0.3)

# Plot 4: Energy convergence with grid size
ax4 = axes[1, 1]

# Test convergence for different grid sizes
grid_sizes = [50, 100, 200, 500, 1000]
E0_values = []
E1_values = []

for N_test in grid_sizes:
    x_test = np.linspace(x_min, x_max, N_test)
    dx_test = x_test[1] - x_test[0]
    V_test = harmonic_potential(x_test, omega)
    
    kp = hbar**2 / (2 * m * dx_test**2)
    main_d = 2 * kp + V_test
    off_d = -kp * np.ones(N_test - 1)
    
    E_test, _ = eigh_tridiagonal(main_d, off_d)
    E0_values.append(E_test[0])
    E1_values.append(E_test[1])

ax4.plot(grid_sizes, E0_values, 'o-', color='blue', linewidth=2, markersize=8, label='E₀ (numerical)')
ax4.plot(grid_sizes, E1_values, 's-', color='red', linewidth=2, markersize=8, label='E₁ (numerical)')
ax4.axhline(y=0.5, color='blue', linestyle='--', alpha=0.5, label='E₀ (exact)')
ax4.axhline(y=1.5, color='red', linestyle='--', alpha=0.5, label='E₁ (exact)')

ax4.set_xlabel('Number of Grid Points')
ax4.set_ylabel('Energy (a.u.)')
ax4.set_title('Convergence of Energy Eigenvalues')
ax4.legend(loc='right', fontsize=9)
ax4.grid(True, alpha=0.3)
ax4.set_xscale('log')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved to 'plot.png'")

## Analysis and Discussion

### Key Observations

1. **Energy Quantization**: The finite difference method accurately reproduces the discrete energy spectrum $E_n = (n + 1/2)\hbar\omega$ of the quantum harmonic oscillator.

2. **Wavefunction Nodes**: The $n$-th eigenstate has exactly $n$ nodes (zero crossings), consistent with Sturm-Liouville theory.

3. **Probability Distribution**: Higher energy states have probability density extending further from the origin, with peaks near the classical turning points.

4. **Convergence**: The numerical accuracy improves with increasing grid resolution. The error scales as $O((\Delta x)^2)$ for the central difference scheme.

### Sources of Error

- **Discretization error**: Finite $\Delta x$ introduces truncation errors
- **Boundary effects**: Fixed boundaries at $\pm x_{max}$ can affect highly excited states
- **Numerical precision**: Finite machine precision limits achievable accuracy

### Extensions

This method can be applied to:
- Anharmonic potentials (double wells, Morse potential)
- Periodic potentials (Kronig-Penney model)
- Time-dependent problems (Crank-Nicolson scheme)

In [ ]:
# Final verification: check orthonormality of eigenstates
print("Orthonormality Check:")
print("="*40)
print("<ψ_i|ψ_j> matrix (should be identity):")
print()

overlap_matrix = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        overlap = np.trapz(wavefunctions[:, i] * wavefunctions[:, j], x)
        overlap_matrix[i, j] = overlap

print(np.array2string(overlap_matrix, precision=6, suppress_small=True))